## Use the right version of Tensorflow


In [0]:
%tensorflow_version 2.x

## Import the important statements

The following code imports the necessary code to run the code in the rest of this Colaboratory.

In [104]:
#@title Call the import statements

import numpy as np
import pandas as pd
import nltk
import re

nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from os import path

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import  Dense, Activation, Dropout,Bidirectional, GlobalMaxPool1D,BatchNormalization, Embedding,LSTM, Flatten


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load Stopwords, Stemmer, and Lemmatizer

In [0]:
sw = stopwords.words('english')
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

## Define the replace_puncts, strip_chars, and puncts

this list and arrays are going to be used in defining the functions

In [0]:
replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

## Define functions that clean, lemmatize, process text, and remove stem words from text

The following code defines three functions:

  * `clean_text`, which convert word to lower case and replace some characters
  * `stem`, steam each word in the given text
  * `lemm`, which will lemmatize word
  * `stopwords1`, which remove the stopwords
  * `text_processing`, which incorporate all the function above into one function that will be used to process text later
  * `clean_length`, which remove words that have length less equal than 2

In [0]:
def clean_text(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
    x = x.strip()
    return x

def stopwords1(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 

def lemm(text):    
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(text)

def clean_length(text):
    text = [word for word in text.split() if len(word) > 2]
    return " ".join(text)

def text_processing(X):
    X=X.apply(stopwords1)
    X=X.apply(clean_text)
    X=X.apply(stemming)
    X=X.apply(lemm)
    X=X.apply(clean_length)
    return X

## Loading Dataset

in this project, we use data from Kaggle :  [Fake and Real News Dasaset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset) 


In [108]:
fake_dataset_url = 'https://raw.githubusercontent.com/Bangkit-2-Jakarta-Team/Fake-News-Detection/master/Datasets/Fake.csv'
true_dataset_url='https://raw.githubusercontent.com/Bangkit-2-Jakarta-Team/Fake-News-Detection/master/Datasets/True.csv'

df_fake = pd.read_csv(fake_dataset_url)
df_true = pd.read_csv(true_dataset_url)
df_fake['is_true'] = 0
df_true['is_true'] = 1

# make ratio of true and fake dataset is same
df_length = min(len(df_fake), len(df_true))
df_fake = df_fake[:df_length]
df_true = df_true[:df_length]

#Merge two dataframe into one
df_data = pd.concat([df_fake,df_true])

#Randomizing data sequence to scatter data
df_data = shuffle(df_data).reset_index(drop=True)

#Showing sample
df_data.sample(5)

,title,text,subject,date,is_true
20135,OBAMA FIGHTS TO KEEP RADICAL AGENDA ALIVE: Ask...,This is a good reminder of how important it is...,left-news,"Jul 2, 2016",0
1077,Bill Maher’s #DebateNight Live Tweetstorm Was...,"It s no secret that Bill Maher, comedian extra...",News,"September 27, 2016",0
11344,Thai immigration police chief says no informat...,BANGKOK (Reuters) - The head of Thailand s imm...,worldnews,"August 25, 2017",1
5634,REPORT: The White House is a ‘Real Dump’…Hundr...,"Gross! As it turns out, the White House is a w...",politics,"Nov 30, 2017",0
14670,Minnesota Senate may halt operations on Dec. 1...,(Reuters) - The Minnesota Senate will furlough...,politicsNews,"November 8, 2017",1


## The Features Used

In [0]:
x = df_data['title']
y = df_data['is_true']

## Clean title feature text

In [0]:
#Clean data using text_processing() function
clean_x = text_processing(x)

In [0]:
def remove_word(text):
  return ' '.join([i for i in text.split() if i not in ['trump']]) #remove trump because trump have a high number in 2 label

clean_x = clean_x.apply(remove_word)

## Split train and test dataset

In [0]:
x_train, x_test, y_train, y_test = train_test_split(clean_x, y, test_size=0.3, random_state = 42)

## Tokenizer title text and label encoder

In [0]:
# Techniques Data
y_test_temp = y_test
max_words = 2000

# cast text to token
tokenize = text.Tokenizer(num_words=max_words)
tokenize.fit_on_texts(x_train) # only fit on train
x_train = tokenize.texts_to_sequences(x_train)
x_test = tokenize.texts_to_sequences(x_test)

# encode label
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

max_sequences= 300
x_train=pad_sequences(x_train,maxlen=max_sequences)
x_test=pad_sequences(x_test,maxlen=max_sequences)

## Neural Network Architecture

In [114]:
#LSTM DTS
embed_size = 128

model = Sequential()
model.add(Embedding(max_words,embed_size,input_length=max_sequences))
model.add(Bidirectional((LSTM(64,return_sequences = True,recurrent_dropout=0.5))))
model.add(Dropout(0.6))
model.add(GlobalMaxPool1D())
model.add(Dense(48, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(y.unique()), activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 128)          256000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 300, 128)          98816     
_________________________________________________________________
dropout_10 (Dropout)         (None, 300, 128)          0         
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 48)                6192      
_________________________________________________________________
dropout_11 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                

## Train Model

In [115]:
batch_size = 64
epochs = 10

callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1),]
    
history3 =model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 409s 1s/step - loss: 0.2716 - accuracy: 0.8904 - val_loss: 0.2552 - val_accuracy: 0.9265
Epoch 2/10
375/375 [==============================] - 401s 1s/step - loss: 0.1381 - accuracy: 0.9491 - val_loss: 0.2238 - val_accuracy: 0.9275
Epoch 3/10
375/375 [==============================] - 404s 1s/step - loss: 0.1117 - accuracy: 0.9576 - val_loss: 0.1878 - val_accuracy: 0.9356
Epoch 4/10
375/375 [==============================] - 402s 1s/step - loss: 0.0961 - accuracy: 0.9642 - val_loss: 0.1652 - val_accuracy: 0.9420
Epoch 5/10
375/375 [==============================] - 401s 1s/step - loss: 0.0815 - accuracy: 0.9690 - val_loss: 0.1580 - val_accuracy: 0.9406
Epoch 6/10
375/375 [==============================] - 405s 1s/step - loss: 0.0714 - accuracy: 0.9736 - val_loss: 0.1622 - val_accuracy: 0.9363
Epoch 7/10
375/375 [==============================] - 401s 1s/step - loss: 0.0619 - accuracy: 0.9773 - val_loss: 0.1566 - val_accuracy: 0.9365

## Evaluate model accuracy

In [116]:
accr1 = model.evaluate(x_test,y_test)

402/402 [==============================] - 44s 109ms/step - loss: 0.1789 - accuracy: 0.9374


## Result

In [35]:
print('AKURASI DARI LSTM \nTest set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr1[0],accr1[1]))

AKURASI DARI LSTM 
Test set
  Loss: 0.172
  Accuracy: 0.941
